# Tier 1: probing each target prefixes

In [1]:
import logging
import pickle
import sys
import uuid

from geoloc_imc_2023.cbg import CBG, get_prefix_from_ip
from geoloc_imc_2023.query_api import get_measurements_from_tag
from geoloc_imc_2023.default import LOG_PATH, RIPE_CREDENTIALS
from geoloc_imc_2023.measurement_utils import (
    load_atlas_anchors,
    load_atlas_probes,
    load_prefix_hitlist,
    save_config_file,
)

# generate measurement UUID
measurement_uuid = uuid.uuid4()
dry_run = True
NB_TARGET = 10
NB_VP = 30

# set logging utilities
logging.basicConfig(
        filename=LOG_PATH / (str(measurement_uuid) + ".log"),
        filemode="a",
        format="%(asctime)s::%(name)s:%(levelname)s:%(message)s",
        datefmt="%H:%M:%S",
        level=logging.DEBUG,
)

logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

logger = logging.getLogger()


## load datasets

In [2]:
targets_per_prefix = load_prefix_hitlist()

# load probes
probes = load_atlas_probes()

# load anchors
anchors = load_atlas_anchors()

## select target and vps dataset

In [3]:
# select targets and vps from anchors
targets= list(anchors.keys())[:NB_TARGET]
vps = {}
for i, probe in enumerate(probes):
    if i >= NB_VP: break
    vps[probe] = probes[probe]

logger.info(f"nb targets: {len(targets)}")
logger.info(f"nb_vps : {len(vps)}")

nb targets: 10
nb_vps : 30


## measure target prefixes

In [4]:
# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes as vps",
    "type" : "target prefix",
    "af" : 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
save_config_file(measurement_config)

cbg = CBG(RIPE_CREDENTIALS)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = cbg.prefix_probing(
    target_prefixes= target_prefixes, 
    vps= vps,
    targets_per_prefix= targets_per_prefix,
    tag= measurement_uuid,
    dry_run=dry_run
)

# save measurement configuration before starting measurement
save_config_file(measurement_config)


starting measurement for target_prefix='213.225.160.0' with ['213.225.160.14', '213.225.160.254', '213.225.160.140'] with 30 vps
starting measurement for target_addr='213.225.160.14' with 30
starting measurement for target_prefix='213.225.160.0' with ['213.225.160.14', '213.225.160.254', '213.225.160.140'] with 30 vps
starting measurement for target_addr='213.225.160.254' with 30
starting measurement for target_prefix='213.225.160.0' with ['213.225.160.14', '213.225.160.254', '213.225.160.140'] with 30 vps
starting measurement for target_addr='213.225.160.140' with 30
starting measurement for target_prefix='145.220.0.0' with ['145.220.0.29', '145.220.0.28', '145.220.0.1'] with 30 vps
starting measurement for target_addr='145.220.0.29' with 30
starting measurement for target_prefix='145.220.0.0' with ['145.220.0.29', '145.220.0.28', '145.220.0.1'] with 30 vps
starting measurement for target_addr='145.220.0.28' with 30
starting measurement for target_prefix='145.220.0.0' with ['145.220.0

## Retrieve prefix probing results 

In [ ]:
measurement_results = get_measurements_from_tag(measurement_config["UUID"])

# test that everything is alright
logging.info(f"nb measurements retrieved: {len(measurement_results)}")
print(f"measurements for {len(measurement_results)} target addr.")
for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10: break
    print(target_addr, ":", len(results))

# save results
out_file = f"../results/{measurement_config['UUID']}.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)
    

# Tier 2: probe each target

In [2]:
import logging
import pickle
import sys
import uuid

from geoloc_imc_2023.cbg import CBG
from geoloc_imc_2023.query_api import get_measurements_from_tag
from geoloc_imc_2023.default import LOG_PATH, RIPE_CREDENTIALS
from geoloc_imc_2023.measurement_utils import (
    load_atlas_anchors,
    load_atlas_probes,
    save_config_file,
)

# generate measurement UUID
measurement_uuid = uuid.uuid4()
dry_run = True
NB_TARGET = 10
NB_VP = 30

In [ ]:
# set logging utilities
logging.basicConfig(
        filename=LOG_PATH / (str(measurement_uuid) + ".log"),
        filemode="a",
        format="%(asctime)s::%(name)s:%(levelname)s:%(message)s",
        datefmt="%H:%M:%S",
        level=logging.DEBUG,
)

logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

In [5]:
logger = logging.getLogger()

## load data

In [6]:
# load probes
probes = load_atlas_probes()

# load anchors
anchors = load_atlas_anchors()

## get targets and vps datasets

In [9]:
# select targets and vps from anchors
targets= list(anchors.keys())[:NB_TARGET]
vps = {}
for i, probe in enumerate(probes):
    if i >= NB_VP: break
    vps[probe] = probes[probe]

logger.info(f"nb targets: {len(targets)}")
logger.info(f"nb_vps : {len(vps)}")

nb targets: 10
nb targets: 10
nb_vps : 30
nb_vps : 30


## measure targets

In [10]:
# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes as vps",
    "type" : "target address",
    "af" : 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
save_config_file(measurement_config)

cbg = CBG(RIPE_CREDENTIALS)

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = cbg.target_probing(
    targets= targets, 
    vps= vps,
    tag= measurement_uuid,
    dry_run=dry_run
)

save_config_file(measurement_config)

starting measurement for target_addr='213.225.160.239' with 30 vps
starting measurement for target_addr='213.225.160.239' with 30 vps
starting measurement for target_addr='145.220.0.55' with 30 vps
starting measurement for target_addr='145.220.0.55' with 30 vps
starting measurement for target_addr='5.28.0.17' with 30 vps
starting measurement for target_addr='5.28.0.17' with 30 vps
starting measurement for target_addr='193.171.255.2' with 30 vps
starting measurement for target_addr='193.171.255.2' with 30 vps
starting measurement for target_addr='192.65.184.54' with 30 vps
starting measurement for target_addr='192.65.184.54' with 30 vps
starting measurement for target_addr='185.42.136.158' with 30 vps
starting measurement for target_addr='185.42.136.158' with 30 vps
starting measurement for target_addr='197.80.104.36' with 30 vps
starting measurement for target_addr='197.80.104.36' with 30 vps
starting measurement for target_addr='200.7.84.24' with 30 vps
starting measurement for target

## retrieve measurement results

In [14]:
measurement_results = get_measurements_from_tag(measurement_config["UUID"])

logger.info(f"measurements for {len(measurement_results)} target addr")
for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10: break
    logger.info(f" {target_addr} : number of measurements {len(results)}")

# save results
out_file = f"../results/{measurement_config['UUID']}.pickle"
logger.info(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

INFO:root:measurements for 10 target addr
INFO:root: 188.94.96.182 : number of measurements 4
INFO:root: 104.167.214.46 : number of measurements 4
INFO:root: 85.236.41.237 : number of measurements 4
INFO:root: 185.97.141.2 : number of measurements 4
INFO:root: 92.38.132.60 : number of measurements 4
INFO:root: 95.217.19.69 : number of measurements 4
INFO:root: 204.9.92.179 : number of measurements 4
INFO:root: 54.39.185.31 : number of measurements 4
INFO:root: 80.79.16.242 : number of measurements 4
INFO:root: 195.32.69.197 : number of measurements 4
INFO:root:../results/db5b34f9-edef-4c08-a0fc-332106015718.pickle


## testbench

In [ ]:
uuid = "2e756f0f-4493-4c6d-898d-e9e87cc2d64b"
uuid = "3987fb12-34d0-4af7-a6bd-dd9fe84b98a5"

uuid = "91d6e4de-8ce6-4fb6-a835-285b0aed82f6"

measurement_results = get_measurements_from_tag(uuid)


In [10]:
logger.info(f"measurements for {len(measurement_results)} target addr")
for i, (target_addr, results) in enumerate(measurement_results.items()):
    logger.info(f" {target_addr} : number of measurements {len(results)}")

measurements for 264 target addr
 213.225.160.14 : number of measurements 773
 213.225.160.159 : number of measurements 776
 213.225.160.254 : number of measurements 774
 145.220.0.1 : number of measurements 773
 145.220.0.28 : number of measurements 689
 145.220.0.29 : number of measurements 775
 5.28.0.14 : number of measurements 773
 5.28.0.6 : number of measurements 774
 5.28.0.13 : number of measurements 774
 193.171.255.2 : number of measurements 774
 193.171.255.1 : number of measurements 649
 193.171.255.9 : number of measurements 773
 192.65.184.69 : number of measurements 772
 192.65.184.42 : number of measurements 772
 192.65.184.154 : number of measurements 774
 185.42.136.149 : number of measurements 776
 185.42.136.3 : number of measurements 772
 185.42.136.1 : number of measurements 776
 197.80.104.33 : number of measurements 773
 197.80.104.35 : number of measurements 774
 197.80.104.36 : number of measurements 774
 200.7.84.3 : number of measurements 763
 200.7.84.120 